In [1]:
import pandas as pd
import os
import sqlite3

In [2]:
os.listdir()

['order_reviews.csv',
 'products.csv',
 'orders.csv',
 'product_category_name_translation.csv',
 'Untitled.ipynb',
 'customer.csv',
 'order_payments.csv',
 '.ipynb_checkpoints',
 'order_items.csv',
 'seller.csv']

In [3]:
df = pd.read_csv('seller.csv')
df.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [4]:
#database
db = '/Users/bimie/Desktop/repos/Dynamic-Analytic-Dashboard-With-Plotly/ecommerce'
sqliteConnection = sqlite3.connect(db)

In [5]:
cursor = sqliteConnection.cursor()

In [12]:
query = '''SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ="table" AND 
    name NOT LIKE "sqlite_%";'''
cursor.execute(query)
result = cursor.fetchall()
df = pd.read_sql(query, sqliteConnection )
df.head(10)

,name
0,customer
1,seller
2,products
3,orders
4,order_items
5,order_payments
6,product_category_name_translation


In [8]:
def run_query(query):
    DATA = []
    cursor.execute(query)
    result = cursor.fetchall()
    for d in pd.read_sql(query, sqliteConnection, chunksize=100):
        DATA.append(d)
    df = pd.concat(DATA)
    return df

In [22]:
query = ''' SELECT pt.product_category_name_english, COUNT(oi.product_id) AS 'number_ordered'  FROM order_items oi
        LEFT JOIN products p on p.id = oi.product_id
        LEFT JOIN product_category_name_translation pt ON p.category_name = pt.product_category_name
         GROUP BY p.category_name
          ORDER BY 2 DESC 
          LIMIT 10'''
run_query(query)

,product_category_name_english,number_ordered
0,bed_bath_table,11115
1,health_beauty,9670
2,sports_leisure,8641
3,furniture_decor,8334
4,computers_accessories,7827
5,housewares,6964
6,watches_gifts,5991
7,telephony,4545
8,garden_tools,4347
9,auto,4235


In [15]:
query = '''SELECT * FROM order_items'''
run_query(query)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
45,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
46,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
47,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
48,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72
